In [1]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import time
import sklearn
from sklearn import preprocessing, cluster
import os
# import cv2

feature_names = ['name', 'order', 'hazType', 'meanXdist', 'meanYdist', 'meanAngle', 'meanSLdist', 'meanXspeed', 'meanYspeed', 'medianXdist', 
          'medianYdist', 'medianAngle', 'medianSLdist', 'medianXspeed', 'medianYspeed', 'stdXdist', 'stdYdist', 'stdAngle', 
          'stdSLdist', 'stdXspeed', 'stdYspeed', 'minXdist', 'minYdist', 'minAngle', 'minSLdist', 'minXspeed', 'minYspeed', 
          'maxXdist', 'maxYdist', 'maxAngle', 'maxSLdist', 'maxXspeed', 'maxYspeed', 'rangeXdist',  'rangeYdist', 'rangeAngle', 
          'rangeSLdist', 'rangeXspeed', 'rangeYspeed', 'skewXdist', 'skewYdist', 'skewAngle', 'skewSLdist', 'skewXspeed', 'skewYspeed', 
          'kurtXdist', 'kurtYdist', 'kurtAngle', 'kurtSLdist', 'kurtXspeed', 'kurtYspeed', 'cvXdist', 'cvYdist', 
          'cvAngle', 'cvSLdist', 'cvXspeed', 'cvYspeed', 'madXdist', 'madYdist', 'madAngle', 'madSLdist', 'madXspeed', 'madYspeed', 
          'action']

feature_names1 = ['name', 'meanXdist', 'meanYdist', 'meanAngle', 'meanSLdist', 'meanXspeed', 'meanYspeed', 'medianXdist', 
          'medianYdist', 'medianAngle', 'medianSLdist', 'medianXspeed', 'medianYspeed', 'stdXdist', 'stdYdist', 'stdAngle', 
          'stdSLdist', 'stdXspeed', 'stdYspeed', 'minXdist', 'minYdist', 'minAngle', 'minSLdist', 'minXspeed', 'minYspeed', 
          'maxXdist', 'maxYdist', 'maxAngle', 'maxSLdist', 'maxXspeed', 'maxYspeed', 'rangeXdist',  'rangeYdist', 'rangeAngle', 
          'rangeSLdist', 'rangeXspeed', 'rangeYspeed', 'skewXdist', 'skewYdist', 'skewAngle', 'skewSLdist', 'skewXspeed', 'skewYspeed', 
          'kurtXdist', 'kurtYdist', 'kurtAngle', 'kurtSLdist', 'kurtXspeed', 'kurtYspeed', 'cvXdist', 'cvYdist', 
          'cvAngle', 'cvSLdist', 'cvXspeed', 'cvYspeed', 'madXdist', 'madYdist', 'madAngle', 'madSLdist', 'madXspeed', 'madYspeed']

dir = "c:/Users/Erika/Desktop/BnE2021/"
os.chdir(dir)
data = pd.read_csv("Nov112021_summaryvariables.csv", header =0, index_col=0, names = feature_names)



In [3]:
# Make a vector of te hazard type before normalizing to use for labelling the clusters 
haz_type = data['hazType']


# #Drop subject number so I can normalize
df = data.drop(['order', 'hazType', 'action'], axis=1)

df1 = preprocessing.normalize(df)
df1 = pd.DataFrame(df1)
data = df1.to_numpy()


In [15]:
#Chose 4 clusters, haven't tried to change this value. The 'Organize' function only works with 4 clusters

K = 4
j_clust = 0
vectors = data

clustering(vectors, K, haz_type)

cluster.KMeans(n_clusters=K, init='k-means++', n_init=10, max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto')

KeyError: 0

In [5]:
#vectors is the data set, K in the number of clusters
def clustering(vectors, K, haz_type): 
    
    #Randomly chose 5 iterations
    iterations = 5
    j_clust = np.zeros((iterations))
    
    #Step 1: Initialize the centroid (call initial_centroid function)
    centroids = initalize_centroid(vectors, K)
    
    #Step 2: Go through all data points and assign to cluster 
    vector_clusters, true_haz_in_cluster_0, true_haz_in_cluster_1, true_haz_in_cluster_2, true_haz_in_cluster_3 = assign_clusters(centroids, vectors, K, haz_type)
    
    #Step 3: Calculate the cost function J, given the first set of centroids and labels
    j_clust[0] = cost_function(centroids, vector_clusters, vectors, K)
    
    #Step 4: Repeat loop for the amount of iterations dictatcted in the assignment 
    for i in range (1, iterations): 

        #Step 4.1: Use the newly clustered data to calculate 'K' new centroids 
        centroids = new_centroids(vectors, vector_clusters, centroids, K)
        
        #Step 4.2: Go through all data points and assign to new cluster 
        vector_clusters, true_haz_in_cluster_0, true_haz_in_cluster_1, true_haz_in_cluster_2, true_haz_in_cluster_3 = assign_clusters(centroids, vectors, K, haz_type)
        
        #Step 4.3: Calculate the cost function J, given the new set of centroids and labels
        j_clust[i] = cost_function(centroids, vector_clusters, vectors, K)
    
    #Step 5: Plot how cost funtion changes through the interations 
    plot_cost_function(j_clust, iterations)
    
    #Step 6: Organize data
    organize(vector_clusters, true_haz_in_cluster_0, true_haz_in_cluster_1, true_haz_in_cluster_2, true_haz_in_cluster_3)
     
    return
    

In [14]:
#STEP 1
def initalize_centroid(vectors,k):
    
    vector_length = len(vectors)
    centroids = np.zeros((k,60))
    
    #Choose random data points for the initial centroids 
    for i in range (0, k):
        random_variable = random.randrange(0, vector_length)
        centroid = vectors[random_variable]
        centroids[i] = centroid      
        
    return(centroids)

In [7]:
#STEP 2 & 4.2
def assign_clusters(centroids, vectors, K, haz_type):
   
    true_haz_in_cluster_0 = []
    true_haz_in_cluster_1 = []
    true_haz_in_cluster_2 = []
    true_haz_in_cluster_3 = []
    
    #First column is cluster number, second is the distance from the vector to the cluster centroid
    distances = np.zeros((K, 2))
    #Cluster number resulting in the smallest distance between vector and current cluster centroid
    clustered_data = np.zeros((len(vectors), 1))
 
    #For each vector, find the closest centroid by choosing smallest euclidean distance 
    for i in range (0, len(vectors)):
        
        #Fill matrix with Euclidean distance to each cluster
        for k in range (0, K):
            distances[k,0] = k + 1
            distances[k,1] = np.sum(np.square(vectors[i] - centroids[k]))
        
        #Sort data so the shortest distance appears at the top, and take the assiociated cluster number 
        sort = distances[np.argsort(distances[:,1])]
        feature_cluster = sort[0,0]
        clustered_data[i] = feature_cluster
        
        #Make 4 vectors of the known hazards to see how they are distributed within the clusters 
        if feature_cluster == 1:
            true_haz_in_cluster_0.append(haz_type[i])
        
        elif feature_cluster == 2:
            true_haz_in_cluster_1.append(haz_type[i])
            
        elif feature_cluster == 3:
            true_haz_in_cluster_2.append(haz_type[i])
        
        elif feature_cluster == 4:
            true_haz_in_cluster_3.append(haz_type[i])
    
    return clustered_data, true_haz_in_cluster_0, true_haz_in_cluster_1, true_haz_in_cluster_2, true_haz_in_cluster_3
            

In [8]:
#STEP 3 & 4.3
def cost_function(centroids, vector_clusters, vectors, K):
    
    j_clust_total = 0
    
    for k in range (0, K):
        j_clust = 0
    
        for i in range (0, len(vectors)):
            
            if vector_clusters[i] == k + 1:
                j_clust = j_clust + np.sum(np.square(vectors[i] - centroids[k]))
            
        j_clust_total = j_clust_total + j_clust
    
    return j_clust_total

In [9]:
#STEP: 4.1
def new_centroids(vectors, vector_clusters, centroids, K):
   
    total = 0
    
    num_vectors = np.zeros((K, 2))
    new_centroids = np.zeros((K,10))
    
    #For each cluster go through all the vectors and determine if they are currently in that cluster
    for k in range (0, K):
        vectors_in_cluster = []
        num_vectors[k,0] = k + 1
        
        for i in range (0, len(vectors)):

            #If they are in that cluster, put them into a matrix 
            if vector_clusters[i] == k + 1:
                vectors_in_cluster.append(vectors[i,:])
                num_vectors[k,1] += 1
        
        #If no vectors are currently in that cluster (not probable), the centroid for the cluster is the same as before        
        if num_vectors[k,1] == 0:
            new_centroids[k,:] = centroids[k,:]   
            
        #If there are vectors in the current cluster, take average of those vectors to make new centroid vector
        else:
            new_centroids[k,:] = np.mean(vectors_in_cluster, axis=0)

    return new_centroids

In [10]:
#STEP 5
def plot_cost_function(j_clust_values, iterations):
    
    plt.figure(figsize=(12, 4)),
    
    for i in range (0, iterations):
        plt.plot(i, j_clust_values[i], 'r.', markersize=14)
        
    plt.xlabel("Iteration Number", fontsize=12)
    plt.ylabel("Cost Value", rotation=90, fontsize=12)
    plt.axis([0, iterations, 0, j_clust_values[0] + 100])
    plt.show()
    
    return

In [11]:
#STEP 6
#Probably a faster anf cleaner way to do this. 

def organize(vector_clusters, true_haz_in_cluster_0, true_haz_in_cluster_1, true_haz_in_cluster_2, true_haz_in_cluster_3):
    
    len1 = len(true_haz_in_cluster_0)
    len2 = len(true_haz_in_cluster_1)
    len3 = len(true_haz_in_cluster_2)
    len4 = len(true_haz_in_cluster_3)
        
    haz1 = np.zeros((4, 1))
    haz2 = np.zeros((4, 1))
    haz3 = np.zeros((4, 1))
    haz4 = np.zeros((4, 1))
    
    ##############CLUSTER 1#####################
    print('The number of vectors in cluster 1 is:', len1)
    
    #Seperate the array of true hazard values that is associated with the cluster
    for i in range (0, len1):
        if true_haz_in_cluster_0[i] == 0:
            haz1[0] += 1
        
        elif true_haz_in_cluster_0[i] == 1:
            haz1[1] += 1
        
        elif true_haz_in_cluster_0[i] == 2:
            haz1[2] += 1
            
        elif true_haz_in_cluster_0[i] == 3:
            haz1[3] += 1
    
    print('There are', haz1[0], 'Instances of hazard 1')
    print('There are', haz1[1], 'Instances of hazard 2')
    print('There are', haz1[2], 'Instances of hazard 3')
    print('There are', haz1[3], 'Instances of hazard 4')
    
    sort1 = haz1.argsort(axis=0)
    highest_haz1 = sort1[3] + 1
    print('The most common hazard in cluster 1 is:', highest_haz1, '\n')
    
    ###############Cluster 2####################
    print('The number of vectors in cluster 2 is:', len2)
    
    for i in range (0, len2):
        if true_haz_in_cluster_1[i] == 0:
            haz2[0] += 1
        
        elif true_haz_in_cluster_1[i] == 1:
            haz2[1] += 1
        
        elif true_haz_in_cluster_1[i] == 2:
            haz2[2] += 1
            
        elif true_haz_in_cluster_1[i] == 3:
            haz2[3] += 1
    
    print('There are', haz2[0], 'Instances of hazard 1')
    print('There are', haz2[1], 'Instances of hazard 2')
    print('There are', haz2[2], 'Instances of hazard 3')
    print('There are', haz2[3], 'Instances of hazard 4')
    
    sort2 = haz2.argsort(axis=0)
    highest_haz2 = sort2[3] + 1
    
    print('The most common hazard in cluster 2 is:', highest_haz2, '\n')
    
    ############ Cluster 3 ######################
    print('The number of vectors in cluster 3 is:', len3)
    
    for i in range (0, len3):
        if true_haz_in_cluster_2[i] == 0:
            haz3[0] += 1
        
        elif true_haz_in_cluster_2[i] == 1:
            haz3[1] += 1
        
        elif true_haz_in_cluster_2[i] == 2:
            haz3[2] += 1
            
        elif true_haz_in_cluster_2[i] == 3:
            haz3[3] += 1
    
    print('There are', haz3[0], 'Instances of hazard 1')
    print('There are', haz3[1], 'Instances of hazard 2')
    print('There are', haz3[2], 'Instances of hazard 3')
    print('There are', haz3[3], 'Instances of hazard 4')
    
    sort3 = haz3.argsort(axis=0)
    highest_haz3 = sort3[3] + 1
    print('The most common hazard in cluster 3 is:', highest_haz3, '\n')
    
    ###########Cluster 4#######################
    print('The number of vectors in cluster 4 is:', len4)
    
    for i in range (0, len4):
        if true_haz_in_cluster_3[i] == 0:
            haz4[0] += 1
        
        elif true_haz_in_cluster_3[i] == 1:
            haz4[1] += 1
        
        elif true_haz_in_cluster_3[i] == 2:
            haz4[2] += 1
            
        elif true_haz_in_cluster_3[i] == 3:
            haz4[3] += 1
    
    print('There are', haz4[0], 'Instances of hazard 1')
    print('There are', haz4[1], 'Instances of hazard 2')
    print('There are', haz4[2], 'Instances of hazard 3')
    print('There are', haz4[3], 'Instances of hazard 4')
    
    sort4 = haz4.argsort(axis=0)
    highest_haz4 = sort4[3] + 1
    print('The most common hazard in cluster 4 is:', highest_haz4)
        

    return

In [12]:
def normalize(df):
    
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        
    return result